# импорты

In [1]:
import boto3
import pandas as pd

from mlds23_authorship_identification.preprocessing import text_splitter

In [2]:
import sys


print(sys.path)

['/Users/dariamishina/Documents/HSE/YearProject/mlds23-authorship-identification/data_preproc/notebooks', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages']


In [3]:
# добавляем в path папку с тем файлом, откуда надо импортировать функции
sys.path.append(
    "/Users/dariamishina/Documents/HSE/YearProject/mlds23-authorship-identification"
)

In [ ]:
# sys.path.remove('/Users/dariamishina/Documents/HSE/YearProject/mlds23-authorship-identification')

In [ ]:
import os

from dotenv import load_dotenv


load_dotenv()

S3_KEY_ID = os.environ.get("S3_KEY_ID")
S3_SECRET_KEY = os.environ.get("S3_SECRET_KEY")
S3_BUCKET = os.environ.get("S3_BUCKET")

# загружаем тексты из s3

In [25]:
session = boto3.session.Session()

In [26]:
s3 = session.client(
    service_name="s3",
    endpoint_url="https://storage.yandexcloud.net",
    aws_access_key_id=S3_KEY_ID,
    aws_secret_access_key=S3_SECRET_KEY,
    region_name="ru-cental1",
)

In [45]:
for key in s3.list_objects(Bucket=S3_BUCKET)["Contents"]:
    print(key["Key"])

dmitry_data/0.pkl
dmitry_data/1.pkl
dmitry_data/2.pkl
dmitry_data/3.pkl
dmitry_data/4.pkl
dmitry_data/5.pkl
dmitry_data/6.pkl
dmitry_data/7.pkl
dmitry_data/8.pkl
dmitry_data/9.pkl
dmitry_data/aleksandr_kuprin_belyj_pudel'.pkl
dmitry_data/aleksandr_kuprin_chudesnyj_doktor.pkl
dmitry_data/aleksandr_kuprin_gambrinus.pkl
dmitry_data/aleksandr_kuprin_granatovyj_braslet.pkl
dmitry_data/aleksandr_kuprin_izumrud.pkl
dmitry_data/aleksandr_kuprin_kust_sireni.pkl
dmitry_data/aleksandr_kuprin_moloh.pkl
dmitry_data/aleksandr_kuprin_osennie_tsvety.pkl
dmitry_data/aleksandr_kuprin_sulamif'.pkl
dmitry_data/aleksandr_kuprin_taper.pkl
dmitry_data/aleksandr_pushkin_arap_petra_velikogo.pkl
dmitry_data/aleksandr_pushkin_dubrovskij.pkl
dmitry_data/aleksandr_pushkin_istorija_sela_gorjuhina.pkl
dmitry_data/aleksandr_pushkin_kapitanskaja_dochka.pkl
dmitry_data/aleksandr_pushkin_pikovaja_dama.pkl
dmitry_data/aleksandr_pushkin_povesti_pokojnogo_ivana_petrovicha_belkina.pkl
dmitry_data/aleksandr_pushkin_puteshest

# загружаем и сразу же сплиттим на токены

Идея от Лены как разбивать произведения

1) Токенизировать nltk/spacy
2) Идем по списку токенов, как только набрали например 550 (это кстати точно гораздо больше, чем 5 предложений) — отрезаем все, которые были после последней запятой/точки/т.д. Т.е. если встреченный токен — пунктуация, то храним его индекс, потом срез до него

In [29]:
total_df = []
for key in s3.list_objects(Bucket=S3_BUCKET)["Contents"]:
    # try:
    #     if "txt" in key["Key"]:  # фильтруем по расширению файла, нужны только txt
    #         file = s3.get_object(Bucket=S3_BUCKET, Key=key["Key"])["Body"]
    #         text = file.read().decode("utf-8")
    #         df = text_splitter(
    #             text, number=3000
    #         )  # вызываем функцию из utils для сплитования
    #         df["book"] = key["Key"]  # добавляем название произведения в отдельный столбец
    #         total_df.append(df)
    # except:
    #     print("не тот файл", key["Key"])
    #     continue
    if "txt" in key["Key"]:  # фильтруем по расширению файла, нужны только txt
        file = s3.get_object(Bucket=S3_BUCKET, Key=key["Key"])["Body"]
        text = file.read().decode("utf-8")
        df = text_splitter(text, number=3000)  # вызываем функцию из utils для сплитования
        df["book"] = key["Key"]  # добавляем название произведения в отдельный столбец
        total_df.append(df)
    else:
        print("не тот файл", key["Key"])
        continue

In [30]:
total_df = pd.concat(total_df)
total_df.reset_index(drop=True, inplace=True)
total_df.to_csv("splitted_df_3000.csv", index=False)

In [31]:
total_df.head()

,target,text,book
0,author_id_06,"Узкими горными тропинками , от одного дачного ...",raw_data/aleksandr_kuprin_belyj_pudel'.txt
1,author_id_06,в вечных скитаниях : пуделя Арто и маленького ...,raw_data/aleksandr_kuprin_belyj_pudel'.txt
2,author_id_06,", когда весь Крым наполняется нарядной , богат...",raw_data/aleksandr_kuprin_belyj_pudel'.txt
3,author_id_06,"махали на них с балконов руками , в третьих пр...",raw_data/aleksandr_kuprin_belyj_pudel'.txt
4,author_id_06,"запыленных кипарисов , похожих на длинные черн...",raw_data/aleksandr_kuprin_belyj_pudel'.txt


In [32]:
len(total_df.text[0].split())

537

In [43]:
DATA_PATH = "/Users/dariamishina/Documents/HSE/YearProject/mlds23-authorship-identification/data_preproc/notebooks/"  # это локальный путь! Куда сохранился мой csv файл
DF_FILE_NAME = "splitted_df_3000.csv"

# загружаем в s3 

In [44]:
s3.upload_file(f"{DATA_PATH}{DF_FILE_NAME}", S3_BUCKET, f"splitted_data/{DF_FILE_NAME}")